# Training Steps

1. Load training data
2. Remove columns with low variance
3. Write function to find N closest points (TODO: optimize)
4. Loop through each point, computing y_hat as a function of other points
5. Add (y_hat - y) ** 2 to the total error function
6. Minimize the total error function w.r.t 

# Eval Steps
1. Load test data
2. For each point, find N closest points
3. Compute y_hat as function of other points

In [2]:
import mnist
x_train, y_train, x_test, y_test = mnist.load()
num_columns = len(x_train[0])
num_rows = len(x_train)
print(num_columns)
print(num_rows)

784
60000


In [4]:
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold = 12200.0) # 12730.0 yields 4 points
x_train_filtered = selector.fit_transform(x_train)
num_columns = len(x_train_filtered[0])
num_rows = len(x_train_filtered)
print(num_columns)
print(num_rows)

69
60000


In [5]:
import sympy
import numpy
n = sympy.symbols('n:10')
y = sympy.symbols(f"y:{num_rows}")
x_y_train = []
x_to_y_map = {}
for idx, x in enumerate(x_train_filtered):
    x_tuple = tuple(x.tolist())
    y_simplified = y[idx]
    if x_tuple in x_to_y_map.keys():
        y_simplified = x_to_y_map[x_tuple]
    else:
        x_to_y_map[x_tuple] = y[idx]

    with_y = numpy.append(x, y_simplified)
    with_c = numpy.append(with_y, 1) # The constant 1 is used to fit a plane to the points
    with_n = numpy.append(with_c, n[y_train[idx]])
    x_y_train.append(with_n)
x_y_train = numpy.array(x_y_train)
print(x_y_train)

[[253 253 253 ... y0 1 n5]
 [54 227 253 ... y1 1 n0]
 [0 0 0 ... y2 1 n4]
 ...
 [0 0 52 ... y59997 1 n5]
 [0 0 62 ... y59998 1 n6]
 [0 0 0 ... y59999 1 n8]]


In [8]:
def FindNClosestUnique(points, t):
    def Distance(x):
        dist = 0
        for idx, x_val in enumerate(x[:-2]):
            dist += (int(t[idx]) - int(x_val)) ** 2
        return dist
    
    sorted_points = sorted(points, key=lambda x: Distance(x))
    n = len(t) - 1 # To fit a plane to the points, we need (dimensions + 1) points. T contains constant and y extra columns.
    closest = []
    i = 0
    while len(closest) < n:
        if not (sorted_points[i] == t).all() and not any((sorted_points[i] == x).all() for x in closest):
            closest.append(sorted_points[i])
        i += 1

    return numpy.array(closest)

def SinglePointError(points, t):
    n_closest = FindNClosestUnique(points[:, :-1], t[:-1])
    det = numpy.insert(n_closest, 0, t[:-1], axis=0)
    idx = numpy.argwhere(numpy.all(det[..., :] == 0, axis=0))
    det = numpy.delete(det, idx, axis=1)

    if len(idx) > 0:
        det = det[:-1*len(idx)]

    det_matrix = sympy.Matrix(det)
    eq = det_matrix.det()
    print(eq)
    soln = sympy.solve(eq, t[-3])
    if len(soln) == 0:
        return 0

    point_error = (soln[0] - t[-1]) ** 2
    return point_error

In [9]:
total_error = 0
for idx, x in enumerate(x_y_train):
    total_error += SinglePointError(x_y_train, x)

    if idx == 10:
        break

print(total_error)

-151766853367531995294707565090529244288072265504899743189084786549846091097207538439281771493523659383510146078217139418457757652269198489596584523137*y0 + 4692958283054071218366769443412211672887421512820503997112426579126390473361724878965790556093615952287117698225705685378210038843674600518598982331*y11396 + 109205875290137685758869196732570126443276094324141007831529192816927113715293531833075124465381860418324477877715328929846333501462633007638433635037*y11729 + 33424103421005247692793643151758142359564699664054448816869627425628875771381101977101832297986661520134302016090051181190651224611626990705527813571*y12568 + 134549265044931313952565390783689238053136220519311557631815730740734902319536624072331526990644993818502158011807034324493338819085016224753794766393*y13459 - 218406703514507137870990756328980847769342759900089040340570157206701295762684078099106299409579965944527607509618844142711846595181703626760649030037*y13497 + 4389082821440785532257615893468870770921796806

In [ ]:
# TODO: minimize total error (i.e. get optimal y values)